In [1]:
%matplotlib inline

from datastack.ml.pipelines import face
from datastack.dbs.rdb import RosettaDBMongo
from datastack.dbs.vdb import HpcVarianceDB
from datastack.tools import facerenderer
from datastack.ml.pipelines.face import models

reload(face)
reload(facerenderer)
reload(models)

Couldn't import dot_parser, loading of dot files will not be possible.


<module 'datastack.ml.pipelines.face.models' from '/Users/rsabatini/Develop/cerebro/datastack/datastack/ml/pipelines/face/models.pyc'>

## Initial parameters

In [2]:
# ----------------------------------------
#   General imports and definitions
# ----------------------------------------

jarvis_host       = "jarvis.hli.io"

rosetta_host      = "rosetta.hli.io"
rosetta_namespace = "hg19"

variantdb_host    = "variantdb.hli.io"
variantdb_port    = "8080"
variantdb_version = "HG19_IsaacVariantCaller_AllSamples_PASS_MAF_001_gVCF"

# ----------------------------------------
#   Models map
# ----------------------------------------

models_to_run                                 = {}
models_to_run["MlPipelineMultivariateRidge"]  = models.MlPipelineMultivariateRidge

# ----------------------------------------
#   Rosetta map
# ----------------------------------------

rosetta_groups                        = {}
rosetta_groups['genome_pc_keys']      = "dynamic.FACEgenomicskinship_new.pc."
rosetta_groups['pheno_keys_gender']   = ["dynamic.FACE.pheno.v1.gender_plusminus"]
rosetta_groups['pheno_keys_age_bmi']  = ["dynamic.FACE.pheno.v1.age", "dynamic.FACE.pheno.v1.bmi"]

# ----------------------------------------
#   VariantDB map
# ----------------------------------------

variantdb_groups                      = {}

# ----------------------------------------
#   Face version
# ----------------------------------------

# Keys: v1, v2, v3, v4, v5
face_version = 'v2'

# ----------------------------------------
#   Sample details
# ----------------------------------------

# Samples not to use in training or testing, https://github.com/hlids/cerebro/wiki/Face-Problems
error_samples   = ["15-0153", "15-0041", "15-0047", "15-0275", \
                  "15-1072", "15-0032", "15-0156", "15-0179", \
                  "15-0107", "15-0191", "15-0787", "15-0107", \
                  "15-0656", "15-0346", "15-0959", "15-0444", "15-0444"]
# ----------------------------------------
#   Medel parameters
# ----------------------------------------

model_rosetta_keys = ["genome_pc_keys", "pheno_keys_gender", "pheno_keys_age_bmi"]
model_rosetta_norm = ['dynamic.FACE.pheno.v1.bmi', 'dynamic.FACE.pheno.v1.age']

model_variant_keys = [] 

train_test_ratio         = 1.0
N_face_PC                = 40
max_samples              = None

heldout_samples          = None
render_heldout           = False
force_test_set           = False

store_test_face_matrixes = False
store_full_model         = True

## Datastack initialization

In [3]:
reload(facerenderer)
frr = facerenderer.FaceRenderer(host=jarvis_host)
frr.initialize(face_version)

rdb = RosettaDBMongo(host=rosetta_host)
rdb.initialize(namespace=rosetta_namespace)

vdb = HpcVarianceDB(host=variantdb_host, port=variantdb_port)
vdb.initialize(version=variantdb_version)

## Data definition

In [4]:
# Rosetta data
rosetta_converted        = {}
for k in model_rosetta_keys:
    if type(rosetta_groups[k])==list:
        rosetta_converted[k]      = []
        for l in rosetta_groups[k]:
            rosetta_converted[k] += rdb.find_keys(l)
    else:
        rosetta_converted[k]      = rdb.find_keys(rosetta_groups[k])

In [5]:
reload(face)
da = face.DatasetAssembler(rdb, vdb, frr, rosetta_map = rosetta_converted, variantdb_map = variantdb_groups)

## Rosetta query

In [6]:
_out = da.prepare_rosetta(model_rosetta_keys,
                              normalize_keys = model_rosetta_norm,
                              max_samples    = max_samples,
                              force_test_set = force_test_set
                              )

In [7]:
# if model_variant_keys is not None and len(model_variant_keys)>0:
#     da.add_variants(model_variant_keys, model_variant_operation, **model_variant_parametrs)

## Train Test preparation

In [8]:
reload(face)
tt = face.TrainTestContainer(da)

In [9]:
tt.create_train_test_split(heldout_samples  = heldout_samples, 
                           remove_samples   = error_samples, 
                           train_test_ratio = train_test_ratio,
                           N_face_PC        = N_face_PC,
                           force_test_set   = force_test_set
                          )

(903, 0)

### Generate the XY set

In [10]:
from datastack.ml.pipelines.face import models
from sklearn import linear_model
from sklearn import neighbors
from sklearn import svm
reload(models)

reload(facerenderer)
frr = facerenderer.FaceRenderer(host=jarvis_host)
frr.initialize(face_version)


In [11]:
reload(models)

models_to_run                                 = {}
models_to_run["MlPipelineMultivariateRidge"]  = models.MlPipelineMultivariateRidge

models_params = {}
models_params["MlPipelineMultivariateRidge"] = {'alpha'         : [ 0.0001, 0.001, 0.01,  0.1, 1, 10, 1000], 
                                                'bmi_key'       : ['dynamic.FACE.pheno.v1.bmi'],
                                                'age_key'       : ['dynamic.FACE.pheno.v1.age'],
                                                'gender_key'    : ['dynamic.FACE.pheno.v1.gender_plusminus'],
                                                'ethnicity_key' : ["dynamic.FACEgenomicskinship_new.pc.{}".format(i+1) for i in range(1000)]}


mname      = "MlPipelineMultivariateRidge"

_params    = {}
if mname in models_params:
    _params = models_params[mname]
    
mlridge      = models_to_run[mname](**_params)
    
m_exec = models.ModelExecutor(mlridge)
m_exec.fit(tt)
# m_exec.compute_model_identification_ranking(kind="color")
# m_exec.compute_model_identification_ranking(kind="depth")

{'est': Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, solver='auto', tol=0.001), 'params': {'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 1000]}}


903 samples,Covariates,SELECT10,R2,c36: R2,c36: MAE,c36: MSE,c40: R2,c40: MAE,c40: MSE,c37: R2,c37: MAE,c37: MSE,c19: R2,c19: MAE,c19: MSE,c18: R2,c18: MAE,c18: MSE,c39: R2,c39: MAE,c39: MSE,c38: R2,c38: MAE,c38: MSE,c13: R2,c13: MAE,c13: MSE,c12: R2,c12: MAE,c12: MSE,c11: R2,c11: MAE,c11: MSE,c10: R2,c10: MAE,c10: MSE,c17: R2,c17: MAE,c17: MSE,c16: R2,c16: MAE,c16: MSE,c15: R2,c15: MAE,c15: MSE,c14: R2,c14: MAE,c14: MSE,c35: R2,c35: MAE,c35: MSE,c9: R2,c9: MAE,c9: MSE,c8: R2,c8: MAE,c8: MSE,c34: R2,c34: MAE,c34: MSE,c3: R2,c3: MAE,c3: MSE,c2: R2,c2: MAE,c2: MSE,c1: R2,c1: MAE,c1: MSE,c7: R2,c7: MAE,c7: MSE,c6: R2,c6: MAE,c6: MSE,c5: R2,c5: MAE,c5: MSE,c4: R2,c4: MAE,c4: MSE,c22: R2,c22: MAE,c22: MSE,c23: R2,c23: MAE,c23: MSE,c20: R2,c20: MAE,c20: MSE,c21: R2,c21: MAE,c21: MSE,c26: R2,c26: MAE,c26: MSE,c27: R2,c27: MAE,c27: MSE,c24: R2,c24: MAE,c24: MSE,c25: R2,c25: MAE,c25: MSE,c31: R2,c31: MAE,c31: MSE,c28: R2,c28: MAE,c28: MSE,c29: R2,c29: MAE,c29: MSE,c30: R2,c30: MAE,c30: MSE,c33: R2,c33: MAE,c33: MSE,c32: R2,c32: MAE,c32: MSE
Mean,,0.000,0.000,0.000,0.048,0.004,0.000,0.043,0.003,0.000,0.046,0.003,0.000,0.089,0.012,0.000,0.095,0.014,0.000,0.044,0.003,0.000,0.044,0.003,0.000,0.135,0.029,0.000,0.155,0.038,0.000,0.158,0.041,0.000,0.153,0.039,0.000,0.104,0.018,0.000,0.108,0.019,0.000,0.113,0.020,0.000,0.130,0.027,0.000,0.048,0.004,0.000,0.186,0.056,0.000,0.225,0.080,0.000,0.051,0.004,0.000,0.589,0.545,0.000,1.049,1.552,0.000,1.423,3.203,0.000,0.259,0.107,0.000,0.255,0.108,0.000,0.287,0.130,0.000,0.405,0.269,0.000,0.081,0.011,0.000,0.077,0.009,0.000,0.088,0.012,0.000,0.085,0.011,0.000,0.067,0.007,0.000,0.063,0.007,0.000,0.071,0.009,0.000,0.070,0.008,0.000,0.057,0.005,0.000,0.061,0.006,0.000,0.063,0.006,0.000,0.058,0.005,0.000,0.052,0.004,0.000,0.053,0.005
Median,,0.000,0.000,-0.003,0.048,0.004,-0.000,0.043,0.003,-0.000,0.046,0.003,-0.002,0.089,0.012,-0.001,0.095,0.014,-0.000,0.044,0.003,-0.000,0.044,0.003,-0.001,0.135,0.029,-0.001,0.155,0.038,-0.000,0.158,0.041,-0.001,0.152,0.039,-0.000,0.104,0.018,-0.001,0.108,0.019,-0.005,0.113,0.020,-0.006,0.130,0.027,-0.000,0.048,0.004,-0.004,0.186,0.057,-0.002,0.225,0.080,-0.000,0.051,0.004,-0.000,0.589,0.545,-0.006,1.047,1.561,-0.002,1.421,3.210,-0.000,0.259,0.107,-0.000,0.255,0.108,-0.000,0.287,0.130,-0.002,0.404,0.270,-0.000,0.081,0.011,-0.000,0.077,0.009,-0.007,0.087,0.012,-0.000,0.085,0.011,-0.002,0.067,0.007,-0.000,0.063,0.007,-0.003,0.071,0.009,-0.001,0.070,0.008,-0.000,0.057,0.005,-0.000,0.061,0.006,-0.001,0.063,0.006,-0.000,0.058,0.005,-0.001,0.052,0.004,-0.000,0.053,0.005
Ridge,Age,0.150,0.013,-0.006,0.048,0.004,-0.008,0.043,0.003,0.004,0.045,0.003,0.018,0.088,0.012,0.017,0.093,0.013,0.002,0.044,0.003,-0.002,0.044,0.003,0.010,0.134,0.029,0.091,0.148,0.034,-0.009,0.159,0.041,0.010,0.152,0.039,0.054,0.102,0.017,-0.005,0.109,0.020,0.015,0.112,0.020,0.057,0.127,0.026,0.006,0.048,0.004,0.049,0.181,0.054,0.002,0.224,0.080,-0.005,0.051,0.004,-0.003,0.590,0.547,0.168,0.931,1.292,0.009,1.424,3.175,-0.004,0.259,0.108,-0.003,0.255,0.108,0.036,0.281,0.125,-0.001,0.405,0.270,0.011,0.081,0.010,-0.006,0.077,0.009,0.009,0.087,0.012,-0.001,0.085,0.011,-0.006,0.067,0.007,0.010,0.063,0.006,0.004,0.071,0.009,-0.005,0.070,0.008,0.014,0.056,0.005,-0.005,0.061,0.006,-0.003,0.063,0.006,0.003,0.058,0.005,-0.005,0.052,0.004,0.000,0.053,0.005
Ridge,Gender,0.199,0.020,-0.005,0.048,0.004,-0.009,0.043,0.003,0.003,0.045,0.003,-0.003,0.089,0.012,-0.006,0.095,0.014,-0.007,0.044,0.003,-0.005,0.044,0.003,0.019,0.133,0.029,0.001,0.155,0.038,-0.006,0.159,0.041,0.008,0.153,0.039,-0.003,0.104,0.018,0.007,0.108,0.019,0.022,0.111,0.020,-0.004,0.131,0.027,-0.001,0.049,0.004,0.001,0.186,0.056,0.034,0.220,0.077,-0.002,0.051,0.004,0.127,0.546,0.476,0.279,0.895,1.119,0.308,1.138,2.216,-0.002,0.260,0.108,-0.001,0.256,0.108,0.064,0.280,0.121,0.015,0.404,0.265,-0.006,0.082,0.011,-0.003,0.077,0.009,-0.004,0.088,0.012,-0.001,0.085,0.011,-0.005,0.068,0.007,-0.002,0.063,0.007,-0.005,0.072,0.009,-0.006,0.070,0.008,-0.005,0.057,0.00

{'est': Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, solver='auto', tol=0.001), 'params': {'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 1000]}}


903 samples,Covariates,SELECT10,R2,c36: R2,c36: MAE,c36: MSE,c40: R2,c40: MAE,c40: MSE,c37: R2,c37: MAE,c37: MSE,c19: R2,c19: MAE,c19: MSE,c18: R2,c18: MAE,c18: MSE,c39: R2,c39: MAE,c39: MSE,c38: R2,c38: MAE,c38: MSE,c13: R2,c13: MAE,c13: MSE,c12: R2,c12: MAE,c12: MSE,c11: R2,c11: MAE,c11: MSE,c10: R2,c10: MAE,c10: MSE,c17: R2,c17: MAE,c17: MSE,c16: R2,c16: MAE,c16: MSE,c15: R2,c15: MAE,c15: MSE,c14: R2,c14: MAE,c14: MSE,c35: R2,c35: MAE,c35: MSE,c9: R2,c9: MAE,c9: MSE,c8: R2,c8: MAE,c8: MSE,c34: R2,c34: MAE,c34: MSE,c3: R2,c3: MAE,c3: MSE,c2: R2,c2: MAE,c2: MSE,c1: R2,c1: MAE,c1: MSE,c7: R2,c7: MAE,c7: MSE,c6: R2,c6: MAE,c6: MSE,c5: R2,c5: MAE,c5: MSE,c4: R2,c4: MAE,c4: MSE,c22: R2,c22: MAE,c22: MSE,c23: R2,c23: MAE,c23: MSE,c20: R2,c20: MAE,c20: MSE,c21: R2,c21: MAE,c21: MSE,c26: R2,c26: MAE,c26: MSE,c27: R2,c27: MAE,c27: MSE,c24: R2,c24: MAE,c24: MSE,c25: R2,c25: MAE,c25: MSE,c31: R2,c31: MAE,c31: MSE,c28: R2,c28: MAE,c28: MSE,c29: R2,c29: MAE,c29: MSE,c30: R2,c30: MAE,c30: MSE,c33: R2,c33: MAE,c33: MSE,c32: R2,c32: MAE,c32: MSE
Mean,,0.000,0.000,0.000,1.435,3.389,0.000,1.277,2.734,0.000,1.361,3.142,0.000,2.127,7.377,0.000,2.093,7.586,0.000,1.279,2.831,0.000,1.307,2.990,0.000,2.645,11.163,0.000,2.628,11.639,0.000,2.857,12.806,0.000,2.977,14.596,0.000,2.275,8.263,0.000,2.342,9.066,0.000,2.434,9.737,0.000,2.504,10.873,0.000,1.434,3.449,0.000,3.309,18.580,0.000,3.528,21.027,0.000,1.463,3.762,0.000,7.563,84.441,0.000,10.169,177.973,0.000,46.078,2636.298,0.000,3.587,21.418,0.000,4.483,35.278,0.000,5.091,41.824,0.000,5.062,44.588,0.000,1.893,6.105,0.000,1.894,5.967,0.000,2.093,7.017,0.000,2.060,6.934,0.000,1.781,5.215,0.000,1.694,4.894,0.000,1.836,5.680,0.000,1.759,5.258,0.000,1.614,4.289,0.000,1.663,4.607,0.000,1.645,4.524,0.000,1.610,4.381,0.000,1.506,3.735,0.000,1.510,3.889
Median,,0.000,0.000,-0.000,1.435,3.389,-0.002,1.275,2.739,-0.001,1.360,3.144,-0.000,2.127,7.377,-0.000,2.093,7.590,-0.000,1.279,2.832,-0.000,1.307,2.990,-0.000,2.644,11.164,-0.001,2.626,11.648,-0.000,2.857,12.807,-0.003,2.971,14.641,-0.000,2.274,8.265,-0.001,2.342,9.072,-0.002,2.432,9.754,-0.001,2.502,10.881,-0.000,1.433,3.450,-0.001,3.308,18.591,-0.001,3.528,21.038,-0.001,1.462,3.765,-0.012,7.516,85.417,-0.002,10.161,178.251,-0.007,46.020,2654.817,-0.000,3.587,21.426,-0.001,4.481,35.299,-0.001,5.087,41.869,-0.003,5.050,44.741,-0.000,1.892,6.108,-0.000,1.893,5.967,-0.000,2.093,7.017,-0.000,2.060,6.934,-0.000,1.781,5.216,-0.000,1.694,4.894,-0.000,1.836,5.680,-0.000,1.759,5.259,-0.001,1.613,4.293,-0.000,1.663,4.607,-0.005,1.641,4.545,-0.002,1.608,4.388,-0.002,1.505,3.742,-0.002,1.508,3.896
Ridge,Age,0.162,0.011,0.022,1.408,3.315,-0.002,1.280,2.739,0.016,1.351,3.093,-0.005,2.132,7.412,0.014,2.091,7.479,0.004,1.284,2.819,-0.003,1.309,2.997,-0.000,2.645,11.163,-0.001,2.627,11.651,0.026,2.809,12.471,0.049,2.881,13.878,0.014,2.258,8.146,-0.004,2.345,9.098,0.001,2.435,9.730,0.004,2.506,10.825,-0.003,1.435,3.460,-0.003,3.311,18.628,-0.002,3.528,21.071,-0.007,1.468,3.787,0.090,7.383,76.837,0.032,9.996,172.259,0.128,41.264,2299.226,0.003,3.587,21.358,0.018,4.417,34.646,0.005,5.091,41.614,0.047,4.983,42.481,-0.004,1.898,6.129,-0.002,1.902,5.980,-0.004,2.097,7.045,-0.006,2.065,6.972,-0.005,1.785,5.242,0.006,1.694,4.863,0.005,1.832,5.650,0.011,1.753,5.198,-0.009,1.621,4.327,-0.001,1.663,4.610,0.010,1.630,4.477,-0.004,1.611,4.397,-0.003,1.508,3.747,0.006,1.507,3.866
Ridge,Gender,0.182,0.018,-0.006,1.438,3.410,-0.002,1.278,2.738,-0.004,1.363,3.154,-0.005,2.132,7.413,0.001,2.087,7.583,-0.006,1.283,2.847,-0.004,1.309,3.001,-0.003,2.644,11.193,0.000,2.634,11.636,0.004,2.848,12.750,0.004,2.978,14.534,0.005,2.262,8.219,-0.005,2.356,9.114,0.001,2.421,9.726,-0.002,2.502,10.891,-0.003,1.435,3.459,-0.002,3.309,18.615,0.002,3.500,20.978,0.006,1.454,3.738,0.648,4.400,29.728,0.020,9.901,174.455,-0.005,46.252,2649.478,-0.004,3.595,21.511,0.096,4.237,31.876,-0.001,5.095,41.865,0.010,5.046,44.131,0.004,1.879,6.083,-0.004,1.892,5.990,-0.005,2.098,7.054,-0.001,2.062,6.941,-0.007,1.787,5.25

In [12]:
reload(models)
m_exec.store(full=True)

In [13]:
m_exec.root_dir

'/Users/rsabatini/Develop/notebooks/facemodel/pipeline/ml.pipelines.face/MlPipelineMultivariateRidge.0752b078-a84b-11e5-9746-3c15c2cc0aae'

In [14]:
reload(models)

model_dir = m_exec.root_dir

d_model = models.MlPipelineMultivariateRidge()
d_model.load(model_dir+"/depth/")
c_model = models.MlPipelineMultivariateRidge()
c_model.load(model_dir+"/color/")


In [15]:
from datastack.tools import facerenderer
frr = facerenderer.FaceRenderer(host=jarvis_host)
frr.initialize(face_version)

In [16]:
d_zero = d_model.predict(tt.train_x.ix[0:2]).values[1]
c_zero = c_model.predict(tt.train_x.ix[0:2]).values[1]

frr.render(x=list(d_zero), z=list(c_zero), quality=5)

In [21]:
import pandas as pd

intake = {}
intake["dynamic.FACE.pheno.v1.gender_plusminus"] = -1
intake["dynamic.FACE.pheno.v1.age"]              = 1.742510
intake["dynamic.FACE.pheno.v1.bmi"]              = -0.012902

df_flat = pd.read_csv("pca_test.csv")
for c in ["pc.{}".format(ii+1) for ii in range(1000)]:
    intake["dynamic.FACEgenomicskinship_new.{}".format(c)] = df_flat.ix[0][c]
    
# for i in range(1000):
#     intake["genomics.kinship.pc.{}".format(i+1)] = read_something
    
indf = pd.DataFrame([intake])
d_zero = d_model.predict(indf).values[0]
c_zero = c_model.predict(indf).values[0]

frr.render(x=list(d_zero), z=list(c_zero), quality=5)

In [24]:
rdb.query(keys=["qc.sample_key"], filters={"qc.sample_name": "187527227"}, outside_index=True)

,qc.sample_key
